In [11]:
import torch
from transformers import AutoTokenizer, EsmModel
import torch.nn as nn
import numpy as np

# some constants
MODEL_NAME = "facebook/esm2_t33_650M_UR50D"
MAX_LENGTH = 1024
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
OUTPUT_SIZE = 1
DROPOUT = 0.25

# UPDATE THIS!
MODEL_PATH = '/home/skrhakv/cryptic-nn/src/650M-version/finetuned-model-650M.pt'

# define the model - if we do not define the model then the loading of the model will fail
class FinetuneESM(nn.Module):
    def __init__(self, esm_model: str) -> None:
        super().__init__()
        self.llm = EsmModel.from_pretrained(esm_model)
        self.dropout = nn.Dropout(DROPOUT)
        self.classifier = nn.Linear(self.llm.config.hidden_size, OUTPUT_SIZE)
        self.plDDT_regressor = nn.Linear(self.llm.config.hidden_size, OUTPUT_SIZE)
        self.distance_regressor = nn.Linear(self.llm.config.hidden_size, OUTPUT_SIZE)

    def forward(self, batch: dict[str, np.ndarray]) -> torch.Tensor:
        input_ids, attention_mask = batch["input_ids"], batch["attention_mask"]
        token_embeddings = self.llm(
            input_ids=input_ids, attention_mask=attention_mask
        ).last_hidden_state
        
        return self.classifier(token_embeddings), self.plDDT_regressor(token_embeddings), self.distance_regressor(token_embeddings)

# load the model
model = torch.load(MODEL_PATH, weights_only=False)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model.eval()

KRAS_sequence = 'GMTEYKLVVVGACGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETSLLDILDTAGQEEYSAMRDQYMRTGEGFLLVFAINNTKSFEDIHHYREQIKRVKDSEDVPMVLVGNKSDLPSRTVDTKQAQDLARSYGIPFIETSAKTRQGVDDAFYTLVREIRKHKEK'

# tokenize the sequence
tokenized_sequences = tokenizer(KRAS_sequence, max_length=MAX_LENGTH, padding='max_length', truncation=True)
tokenized_sequences = {k: torch.tensor([v]).to(DEVICE) for k,v in tokenized_sequences.items()}

# predict
output, _, _ = model(tokenized_sequences)
output = output.flatten()

mask = (tokenized_sequences['attention_mask'] == 1).flatten()

output = torch.sigmoid(output[mask][1:-1]).detach().cpu().numpy()
print(output)


[0.1365  0.2307  0.2751  0.3447  0.3547  0.565   0.3354  0.751   0.3079
 0.6426  0.5156  0.8257  0.942   0.869   0.9253  0.7485  0.801   0.816
 0.7183  0.446   0.2441  0.4717  0.4849  0.2822  0.4011  0.3726  0.2496
 0.588   0.711   0.752   0.645   0.714   0.7246  0.5537  0.7036  0.6147
 0.639   0.61    0.543   0.6655  0.5884  0.564   0.4265  0.2524  0.2012
 0.1455  0.2242  0.2009  0.1746  0.1746  0.12103 0.1559  0.2203  0.2106
 0.33    0.4504  0.7163  0.767   0.8545  0.772   0.6387  0.8135  0.6265
 0.4731  0.4404  0.3525  0.313   0.6807  0.6543  0.3076  0.466   0.75
 0.6245  0.4763  0.3684  0.3962  0.3616  0.09686 0.4937  0.12305 0.1226
 0.09503 0.1968  0.2615  0.277   0.3704  0.277   0.2325  0.2524  0.1844
 0.1643  0.2118  0.2246  0.1383  0.2334  0.34    0.3284  0.2632  0.2534
 0.5117  0.3667  0.2603  0.545   0.725   0.6895  0.371   0.302   0.2874
 0.2437  0.258   0.2346  0.207   0.11475 0.0863  0.1532  0.10486 0.5933
 0.9097  0.5107  0.689   0.724   0.316   0.1787  0.2964  0.2101  0.